# Parquet to Dask
load a parquet dataset into as dask cluster

Once the client connects to the dask cluster we can check for package consistency by calling `dask_client.get_versions(check=True)`.  Scheduler and worker
nodes are guaranteed to have the same package versions in `mlrun` since mismatches would likely cause serialization failures.  The client (in this notebook or in some other process) may have different versions and this mismatch is **less likely** to cause issues.  The following installs can be performed to ensure this mismatch becomes **unlikely** (as of this writing). **This may need to be done periodically as the dockerfiles refresh, unless you specifiy an image tag.**

    !python -m pip install --upgrade pip
    !python -m pip uninstall -y cmake
    !python -m pip install cmake
    !python -m pip install scikit-build

    !python -m pip install lz4
    !python -m pip install git+https://github.com/Blosc/python-blosc.git@v1.8.3
    !python -m pip install cloudpickle==1.2.2

    !python -m pip install numpy==1.17.4

## clean up
delete stale pods

In [1]:
# !mlrun clean -p -r

In [2]:
import mlrun

mlrun.mlconf.dbpath = 'http://mlrun-api:8080'

In [3]:
# replace with your app node ip
mlrun.mlconf.remote_host = '18.223.249.136' 

## parameters


In [4]:
FUNCTION           = 'parquet_to_dask'                                # python function file name
DESCRIPTION        = 'load parquet dataset into a dask cluster'       

IMAGE              = 'yjbds/mlrun-dask:dev'                           # custom image
JOB_KIND           = 'dask'                                           

TASK_NAME          = 'user-task-parq-to-dask'                         # whatever

ARTIFACTS_PATH     = '/User/repos/demos/dask/artifacts'
YAML_PATH          = '/User/repos/demos/dask/yaml'

DATA_PATH          = '/User/repos/demos/dask/dataset/partitions'
DOWNSAMPLE         = 0.02
PARTITION_COLS     = ['Year', 'Month']
PARTITION_COLS     = None

DASK_SHARDS        = 8
DASK_THREADS_PER   = 8
MEMORY_LIMIT       = '5GB'

DASK_KEY           = 'airlines'

## load and configure function

In [5]:
FUNC_PY = '/User/repos/demos/dask/code/parquet-to-dask.py'
FUNC_YAML = '/User/repos/demos/dask/yaml/parquet-to-dask.yaml'

#### some of the following cells can be commented out if this notebook has already been run.

In [7]:
# load function from a local Python file
parq2dask = mlrun.new_function(command=FUNC_PY, 
                               image=IMAGE,
                               kind=JOB_KIND)

parq2dask.spec.remote = True
parq2dask.spec.replicas = DASK_SHARDS 
parq2dask.spec.max_replicas = DASK_SHARDS
parq2dask.spec.service_type = 'NodePort'
parq2dask.spec.image_pull_policy = 'Always'
parq2dask.spec.build.image = IMAGE

parq2dask.export(FUNC_YAML)
# parq2dask = mlrun.import_function(FUNC_YAML)

parq2dask.apply(mlrun.mount_v3io())
parq2dask.deploy()

# create and run the task
parq_to_dask_task = mlrun.NewTask(
    TASK_NAME, 
    handler=FUNCTION,  
    params={
        'parquet_url'      : DATA_PATH,
        'sample'           : DOWNSAMPLE,
        'shards'           : DASK_SHARDS,
        'threads_per'      : DASK_THREADS_PER,
        'memory_limit'     : MEMORY_LIMIT,
        'dask_key'         : DASK_KEY,
        'target_path'      : ARTIFACTS_PATH})

In [8]:
# run
rn = parq2dask.run(parq_to_dask_task)

[mlrun] 2020-02-12 11:34:40,324 starting run user-task-parq-to-dask uid=78ee58e2b6cd4007bb141fe1016be3e2  -> http://mlrun-api:8080
[mlrun] 2020-02-12 11:34:41,751 saving function: parquet-to-dask, tag: latest
[mlrun] 2020-02-12 11:34:49,688 using remote dask scheduler (mlrun-parquet-to-dask-c5cb176f-1) at: 18.223.249.136:30245
[mlrun] 2020-02-12 11:34:49,689 remote dashboard (node) port: 18.223.249.136:30796
[mlrun] 2020-02-12 11:34:49,690 found cluster...
[mlrun] 2020-02-12 11:34:49,690 <Client: 'tcp://10.233.64.49:8786' processes=0 threads=0, memory=0 B>
[mlrun] 2020-02-12 11:37:05,225 column header ['Year' 'Month' 'DayofMonth' 'DayOfWeek' 'CRSDepTime' 'UniqueCarrier'
 'ArrDelay' 'Origin' 'Dest' 'Distance']
[mlrun] 2020-02-12 11:37:05,932 log artifact scheduler at /User/repos/demos/dask/artifacts/scheduler.json, size: None, db: Y

[mlrun] 2020-02-12 11:37:05,963 run ended with state 


/User/.pythonlibs/jupyter-1/lib/python3.6/site-packages/mlrun/render.py:324: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...6be3e2,0,Feb 12 11:34:41,completed,user-task-parq-to-dask,kind=daskowner=adminhost=jupyter-1-56795c755c-bss6b,,parquet_url=/User/repos/demos/dask/dataset/partitionssample=0.02shards=8threads_per=8memory_limit=5GBdask_key=airlinestarget_path=/User/repos/demos/dask/artifacts,,scheduler


to track results use .show() or .logs() or in CLI: 
!mlrun get run 78ee58e2b6cd4007bb141fe1016be3e2  , !mlrun logs 78ee58e2b6cd4007bb141fe1016be3e2 
[mlrun] 2020-02-12 11:37:06,038 run executed, status=completed


### the scheduler artifact can be shared

In [9]:
print(rn.metadata.to_yaml())

uid: 78ee58e2b6cd4007bb141fe1016be3e2
name: user-task-parq-to-dask
project: ''
labels:
  kind: dask
  owner: admin
  host: jupyter-1-56795c755c-bss6b
iteration: 0



In [10]:
rn.metadata.uid

'78ee58e2b6cd4007bb141fe1016be3e2'

In [11]:
rn.outputs['scheduler']

'/User/repos/demos/dask/artifacts/scheduler.json'

#### What's the scheduler address?

In [12]:
import json
json.load(open(rn.outputs['scheduler']))

{'type': 'Scheduler',
 'id': 'Scheduler-dbd2e04a-26e1-4044-8e7b-4c496c7f4c73',
 'address': 'tcp://10.233.64.49:8786',
 'services': {},
 'workers': {'tcp://10.233.64.52:37684': {'type': 'Worker',
   'id': 'tcp://10.233.64.52:37684',
   'host': '10.233.64.52',
   'resources': {},
   'local_directory': '/worker-76a25anc',
   'name': 'tcp://10.233.64.52:37684',
   'nthreads': 1,
   'memory_limit': 4109136640,
   'last_seen': 1581507425.4797685,
   'services': {},
   'metrics': {'cpu': 0.0,
    'memory': 91029504,
    'time': 1581507424.979225,
    'read_bytes': 286.54955592820323,
    'write_bytes': 1494.8669141429343,
    'num_fds': 22,
    'executing': 0,
    'in_memory': 0,
    'ready': 0,
    'in_flight': 0,
    'bandwidth': {'total': 100000000, 'workers': {}, 'types': {}}},
   'nanny': 'tcp://10.233.64.52:45678'},
  'tcp://10.233.64.53:37738': {'type': 'Worker',
   'id': 'tcp://10.233.64.53:37738',
   'host': '10.233.64.53',
   'resources': {},
   'local_directory': '/worker-ckpwiaix'

### notebook cluster

Let's load the scheduler file into a cluster in this notebook using our MLRun artifact:

In [28]:
# import dask
# import dask.dataframe as dd
# from dask.distributed import Client, LocalCluster

#client = Client(scheduler_file=rn.outputs['scheduler'])

#df = client.get_dataset(DASK_KEY)

### create a component 'on the fly' to summarise the table

The nice thing about having a dask cluster loaded with all your data is that you can write _quick and dirty_ jobs either in your notebook, a local file, or a github repo. Here we use a local file:

In [29]:
# write up function in local directory
summ = mlrun.new_function(command='/User/repos/demos/dask/code/describe.py', 
                          image=IMAGE,
                          kind='job')

summ.spec.build.image = IMAGE

summ.export('/User/repos/demos/dask/yaml/describe.yaml')

[mlrun] 2020-02-12 09:02:26,076 function spec saved to path: /User/repos/demos/dask/yaml/describe.yaml


In [30]:
summ.apply(mlrun.mount_v3io())

summ.deploy(skip_deployed=True, with_mlrun=False)

# create the task
summ_task = mlrun.NewTask(
    'user-task-my-sum', 
    handler='table_summary',  
    params={
        'dask_key'   :  DASK_KEY,
        'dask_client': 'scheduler.json',
        'target_path':  ARTIFACTS_PATH,
        'name'       : 'table-summary.csv',
        'key'        : 'table-summary'})

# run
rn2 = summ.run(summ_task)

rn2.outputs

[mlrun] 2020-02-12 09:02:28,656 starting run user-task-my-sum uid=933d1d8ee3bc4d51b708ad53037967cc  -> http://mlrun-api:8080
[mlrun] 2020-02-12 09:02:28,727 Job is running in the background, pod: user-task-my-sum-msr79
[mlrun] 2020-02-12 09:10:14,826 log artifact table-summary at /User/repos/demos/dask/artifacts/table-summary.csv, size: None, db: Y

[mlrun] 2020-02-12 09:10:14,863 run executed, status=completed
final state: succeeded


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...7967cc,0,Feb 12 09:02:38,completed,describe,host=user-task-my-sum-msr79kind=jobowner=admin,,dask_client=scheduler.jsondask_key=airlineskey=table-summaryname=table-summary.csvtarget_path=/User/repos/demos/dask/artifacts,,table-summary


to track results use .show() or .logs() or in CLI: 
!mlrun get run 933d1d8ee3bc4d51b708ad53037967cc  , !mlrun logs 933d1d8ee3bc4d51b708ad53037967cc 
[mlrun] 2020-02-12 09:10:20,165 run executed, status=completed


{'table-summary': '/User/repos/demos/dask/artifacts/table-summary.csv'}

### tests

In [31]:
import pandas as pd
pd.read_csv(rn2.outputs['table-summary'])

,Year,Month,DayofMonth,DayOfWeek,CRSDepTime,ArrDelay,Distance
0,2.470699e+06,2.470699e+06,2.470699e+06,2.470699e+06,2.470699e+06,2.418866e+06,2.466694e+06
1,1.998624e+03,6.553830e+00,1.572528e+01,3.943938e+00,1.334650e+03,7.067391e+00,7.021194e+02
2,6.226767e+00,3.444383e+00,8.784255e+00,1.990817e+00,4.758579e+02,3.079673e+01,5.513512e+02
3,1.987000e+03,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-1.290000e+03,1.100000e+01
4,1.995000e+03,5.000000e+00,1.100000e+01,3.000000e+00,1.120750e+03,-4.000000e+00,4.050000e+02
5,2.003000e+03,9.000000e+00,2.000000e+01,5.000000e+00,1.527500e+03,4.500000e+00,6.965000e+02
6,2.008000e+03,1.200000e+01,3.100000e+01,7.000000e+00,2.050000e+03,6.800000e+01,2.527000e+03
7,2.008000e+03,1.200000e+01,3.100000e+01,7.000000e+00,2.400000e+03,1.446000e+03,4.983000e+03
